In [ ]:
!nvidia-smi

Fri Jul 22 00:43:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 24.44 MiB | 27.38 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [ ]:
! pip -q install timm

     |████████████████████████████████| 512 kB 4.6 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
gc.enable()
import sys
import glob
import math
import time
import random

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

from datetime import datetime
from tqdm.auto import tqdm as tqdmp
from tqdm.autonotebook import tqdm as tqdm
tqdmp.pandas()

# Augmentation
#import albumentations as A
#from albumentations.pytorch import ToTensorV2

# Model
import timm
# import lightly

# TTA
# import ttach as tta

In [ ]:
class CFG:
  def __init__(self):

      self.wandb = False
      self.debug = False
      self.early_stop_flg = True
      self.early_stop_round = 4
      self.exp_name = '064_exp'
      self.seed = 55
      self.n_fold = 4
      self.num_workers = 1
      self.model_name = 'vit_base'
      self.mode = 'Classification'
      self.padding = False
      self.multitask = False
      self.SSL = False
      self.size = 384
      self.fold = []

      self.epochs = 15
      self.batch_size = 4
      self.lr = 1e-4
      self.weight_decay = 1e-5
      self.scheduler = 'scheduler:CosineAneelingLR'
      self.eta_min = 0
      self.T_max = self.epochs
      self.MIXUP = False
      self.TTA = False
      self.alpha = 1.0
      self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      self.n_tta = 10
      self.accumulation_steps = 8
      self.N_MSD = 10
      self.MIXOUT = 0.5

CONFIG = CFG()

In [ ]:
def seed_everything(seed:int==42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CONFIG.seed)

In [ ]:
INPUT_DIR = "/content/drive/MyDrive/nihonga/data/"

EXP_NAME = f'{CONFIG.exp_name}'
OUTPUT_DIR = f'/content/drive/MyDrive/nihonga/interim/{EXP_NAME}/'
MODEL_DIR = f'/content/drive/MyDrive/nihonga/model/{EXP_NAME}/'
OOF_DIR = f'/content/drive/MyDrive/nihonga/oof/{EXP_NAME}/'
SUB_DIR = f'/content/drive/MyDrive/nihonga/sub/{EXP_NAME}/'

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OOF_DIR, exist_ok=True)
os.makedirs(SUB_DIR, exist_ok=True)

print(f'INPUT_DIR: {INPUT_DIR}')
print(f'OUTPUT_DIR: {OUTPUT_DIR}')
print(f'MODEL_DIR: {MODEL_DIR}')
print(f'OOF_DIR: {OOF_DIR}')
print(f'SUB_DIR: {SUB_DIR}')

INPUT_DIR: /content/drive/MyDrive/nihonga/data/
OUTPUT_DIR: /content/drive/MyDrive/nihonga/interim/064_exp/
MODEL_DIR: /content/drive/MyDrive/nihonga/model/064_exp/
OOF_DIR: /content/drive/MyDrive/nihonga/oof/064_exp/
SUB_DIR: /content/drive/MyDrive/nihonga/sub/064_exp/


In [ ]:
train_img = np.load(os.path.join(INPUT_DIR,
                                'train_data.npz'))
test_img = np.load(os.path.join(INPUT_DIR,
                                'test_data.npz'))
sub_df = pd.read_csv(os.path.join(INPUT_DIR,
                                'submission.csv'))

In [ ]:
#!mkdir -p /content/drive/MyDrive/nihonga/data/train/0
#for i in tqdm(range(len(train_img['arr_0']))):

#    img = Image.fromarray(train_img['arr_0'][i])
#    img.save(f"/content/drive/MyDrive/nihonga/data/train/0/{i:04d}.png")


In [ ]:
! cd /content/dino/ && python -m torch.distributed.launch --nproc_per_node=1 main_dino_064_exp.py \
--arch 'vit_base' \
--data_path='/content/drive/MyDrive/nihonga/data' \
--output_dir='/content/drive/MyDrive/nihonga/model/064_exp' \
--epochs=500 \
--saveckp_freq=100 \
--batch_size_per_gpu=32

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Downloading: "https://github.com/facebookresearch/xcit/zipball/main" to /root/.cache/torch/hub/main.zip
| distributed init (rank 0): env://
git:
  sha: cb711401860da580817918b9167ed73e3eef3dcf, status: clean, branch: main

arch: vit_base
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/nihonga/data
dist_url: env://
drop_path_rate: 0.1
epochs: 500
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentu